In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import math

In [ ]:
# we define fisrt the Gaussian function we want to fit
# it has to have the form model(x,a,b,c,d,..) where 
# xdata are the coordinates and a,b,c,d are the parameters
def model(xdata, xc, yc):
    x,y = xdata
    im = np.exp(-0.5*((x-xc)**2+(y-yc)**2))
    return im

# let's define a 2D grid on which to evaluate the model 
xi, yi = np.linspace(-n/2,n/2,n), np.linspace(-n/2,n/2,n)
x,y = np.meshgrid(xi,yi)

# we stacks the two flatten array in a single array 
xdata = np.vstack((x.ravel(), y.ravel()))
# we now generate the image
p0 = np.array([0,0])
ydata = model(xdata, p0[0], p0[1]) + 0.02 * np.random.randn(n*n)

# let's display the simulated Gaussian spot
#plt.imshow(ydata.reshape(n,n))

# We can now fit the spot using a non-linear curve fitting procedure
popt, pcov = curve_fit(model, xdata, ydata.ravel(), p0)

# and display the estimated model:
#plt.imshow(model(xdata,popt[0],popt[1]).reshape(n,n))
fig, ax = plt.subplots(ncols=2, figsize=(10, 5))
ax[0].imshow(ydata.reshape(n,n))
ax[1].imshow(model(xdata,popt[0],popt[1]).reshape(n,n))

# and the 2 parameters and their accuracy
print(f'The spot is centered in {popt[0]:.4f},{popt[1]:.4f} at +/- {math.sqrt(pcov[0,0]):.4f},{math.sqrt(pcov[1,1]):.4f} s.d.')



In [ ]:
from skimage import filters
from scipy.optimize import curve_fit
from scipy.stats import norm
from scipy.ndimage import maximum_filter
from skimage.morphology import disk

def get_locations(img, scale=0.75, pvalue=1e-3):
  """Get 2D spot locations on a single frame"""
  score = filters.difference_of_gaussians(img,0.75,3)
  imax = (score >= maximum_filter(score, size=3))
  bw = score > (score.mean() + norm.ppf(1.0-pvalue) * score.std())
  bw[:,0] = 0
  bw[:,-1] = 0
  bw[0,:] = 0
  bw[-1,:] = 0
  coords = np.fliplr(np.stack(np.nonzero(np.logical_and(bw,imax)),axis=1))
  return coords

import math
from scipy.optimize import curve_fit

def model(xdata, xc, yc, background, amplitude):
  """2D Gaussian model"""
    x,y = xdata
    im = background + amplitude * np.exp(-0.5*((x-xc)**2+(y-yc)**2))
    return im

def fit_spot(model, img, coordinate, size):
  """fit model to the image at coordinates in a (size,size) crop
  Parameter:
  model : function (xdata,x,y,a,b)
  img   : 2d numpy array
  coordinates : x,y coordinate
  size : size of the crop on which is made the fit
  """
  x0 = coordinate[0]-(size//2)
  y0 = coordinate[1]-(size//2)
  x1 = coordinate[0]+(size//2)
  y1 = coordinate[1]+(size//2)
  if x0 > 0 and y0 > 0 and x0 < img.shape[1] and y0 < img.shape[0]:
    x,y = np.meshgrid(np.arange(x0,x1),np.arange(y0,y1))
    xdata = np.vstack((x.ravel(), y.ravel()))
    ydata = img[y0:y1,x0:x1].astype(np.float64)
    p0 = np.array([coordinate[0], coordinate[1], ydata.min(), np.ptp(ydata)],dtype=np.float64)
    try:
      popt, pcov = curve_fit(model, xdata, ydata.ravel(), p0)
    except:
      popt = np.array([coordinate[0], coordinate[1], img.min(), np.ptp(img)], dtype=np.float64)
  else:
    popt = np.array([coordinate[0], coordinate[1], img.min(), np.ptp(img)], dtype=np.float64)
  return popt

from scipy.spatial import distance

def nnlink(coords, max_speed=1):
  """Link coordinates from frame to frame
  Parameter
  ---------
  coords : array with t,x,y input
  max_speed : maximum linking speed/distance 
  Return
  ------
  list of links as (N,2) array
  """
  links = []
  times = np.unique(coords[:,0])
  for t in range(int(times.max()-2)):
    # extract slices corresponding to t and t+1
    id1 = np.nonzero(coords[:,0] == t)[0]
    xy1 = coords[id1,1:3]
    id2 = np.nonzero(coords[:,0] == t+1)[0]
    xy2 = coords[id2,1:3]
    # pair wise distance
    d = distance.cdist(xy1,xy2,'euclidean')
    # find the min distance
    k = np.argmin(d,axis=1)
    # check if the min distance < max_speed
    idx = d[np.arange(xy1.shape[0]), k] < max_speed
    # append the matching pair of indices
    links.append(np.vstack([   id1[idx] , id2[k[idx]]  ]))
  return np.squeeze(np.transpose(np.hstack(links)))

def label_graph(E,V):
  """Label the vertices of the graph following one track at a time
  Parameter
  ---------
  E : edges as a (Ne,2) array
  V : vertices as a (Nv,) array
  Returns
  -------
  labels for each vertices as (Nv,) array
  """
  labels = -np.ones(V.shape[0],dtype=np.int64)
  current_label = -1
  while np.any(labels < 0):
    # new starting point
    k = np.argmin(labels)
    current_label = current_label + 1
    labels[k] = current_label
    # set of vertices linked to k
    idx = E[E[:,0]==k,1]
    S = idx[labels[idx].flatten()<0]
    while S.shape[0] > 0:
      k = S[0]
      # color the new vertex
      labels[k] = current_label
      # get the new set of links starting from k
      idx = E[E[:,0]==k,1]
      S = idx[labels[idx].flatten()<0]
  return labels



txyba = []
for t,frame in enumerate(img):
  coords = get_locations(frame)
  coords_subpx = np.stack([fit_spot(model, frame, p, 5) for p in coords])
  time = t*np.ones((coords_subpx.shape[0],1))
  txyba.append(  np.hstack( [time, coords_subpx])  )

coords_sp = np.vstack(txyba)
lnk = nnlink(coords_sp,10)


# labels the graph
labels = label_graph(lnk,coords_sp)

# create tracks from labels
tracks = [{'label':k, 'indices':np.sort(np.nonzero(labels==k))} for k in np.unique(labels)]

# filter out short tracks
tracks = [x for x in tracks if x['indices'].size > 10]

# represent the tracks in 3D
fig = plt.figure()
ax = plt.axes(projection='3d')
for n in range(len(tracks)):
  idx = tracks[n]['indices'].flatten()
  ax.plot3D(coords_sp[idx,1],coords_sp[idx,2],coords_sp[idx,0])
ax.set_xlabel('x-axis')
ax.set_ylabel('y-axis')
ax.set_zlabel('time')
plt.show();


